In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import (Conv2D, Lambda, LeakyReLU, Dense, Input, add, concatenate, Flatten, BatchNormalization, PReLU, UpSampling2D)
from keras.applications.vgg19 import VGG19, preprocess_input

In [ ]:
# --- Helper Function to Convert BGR -> RGB for Display ---
def bgr_to_rgb(image):
    """Convert a BGR image (H,W,3) to RGB for display purposes."""
    return image[..., ::-1]

In [ ]:
# --- Learning Rate Schedule ---
def lr_schedule(epoch):
    if epoch < 15: return 1e-5  # Warmup
    elif epoch < 50: return 1e-4
    else: return 1e-4 * (0.95 ** (epoch//10))
        
# Initialize optimizers 
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
disc_optimizer = tf.keras.optimizers.Adam(1e-5)

In [ ]:
# SRGAN Model Components

def residual_block(x):
    res = Conv2D(64, 3, padding='same')(x)
    res = PReLU(shared_axes=[1,2])(res)  # Removed BatchNorm
    res = Conv2D(64, 3, padding='same')(res)
    return add([x, res])

def srgan_generator():
    inputs = Input((64,64,3))
    x = Conv2D(64, 3, padding='same')(inputs)
    x = PReLU(shared_axes=[1,2])(x)
    temp = x
    
    # 16 Residual Blocks
    for _ in range(16):
        x = residual_block(x)
    
    x = Conv2D(64, 3, padding='same')(x)
    x = add([x, temp])
    
    # Upsampling
    x = UpSampling2D()(x)
    x = Conv2D(256, 3, padding='same')(x)
    x = PReLU(shared_axes=[1,2])(x)
    
    x = UpSampling2D()(x)
    x = Conv2D(256, 3, padding='same')(x)
    x = PReLU(shared_axes=[1,2])(x)
    
    outputs = Conv2D(3, 3, padding='same', activation='sigmoid')(x)
    return Model(inputs, outputs)

def srgan_discriminator():
    inputs = Input((256, 256, 3))
    x = Conv2D(64, 3, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)  # Use LeakyReLU layer from Keras
    
    # Feature extraction
    for filters in [64, 128, 256, 512]:
        x = Conv2D(filters, 3, strides=2, padding='same')(x)
        x = LeakyReLU(alpha=0.2)(x)  # Use LeakyReLU layer here too
        x = Conv2D(filters, 3, padding='same')(x)
        x = LeakyReLU(alpha=0.2)(x)  # Use LeakyReLU layer here as well
    
    # Decision layer
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = LeakyReLU(alpha=0.2)(x)  # LeakyReLU for dense layer
    outputs = Dense(1)(x)  # No sigmoid
    return Model(inputs, outputs)

In [ ]:
# VGG19 Feature Extractor: 
def build_vgg():
    vgg = VGG19(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
    
    # Specify the layer from which you want to extract features
    block5_conv4 = vgg.get_layer("block5_conv4").output  # Deeper features
    
    # Create a new model with the original input and the new output
    vgg_model = Model(inputs=vgg.input, outputs=block5_conv4)
    
    return vgg_model

In [ ]:
# Combined model for perceptual loss
def create_comb(gen_model, disc_model, vgg, lr_ip, hr_ip):
    gen_img = gen_model(lr_ip)
    gen_features = vgg(gen_img)
    disc_model.trainable = False
    validity = disc_model(gen_img)
    return Model(inputs=[lr_ip, hr_ip], outputs=[validity, gen_features])

In [ ]:
# Data Loading
n = 5000  # Number of images to use
lr_path = "/kaggle/input/sr-data/DATASET/AUG/LR_Aug"
hr_path = "/kaggle/input/sr-data/DATASET/AUG/HR_Aug"

lr_list = os.listdir(lr_path)[:n]
hr_list = os.listdir(hr_path)[:n]

lr_images = []
hr_images = []

# Load images and keep them in BGR (for VGG)
for img in lr_list:
    img_path = os.path.join(lr_path, img)
    img_lr = cv2.imread(img_path)  # BGR format
    lr_images.append(img_lr)

for img in hr_list:
    img_path = os.path.join(hr_path, img)
    img_hr = cv2.imread(img_path)  # BGR format
    hr_images.append(img_hr)

lr_images = np.array(lr_images, dtype=np.float32) / 255.0
hr_images = np.array(hr_images, dtype=np.float32) / 255.0

print(f"LR shape: {lr_images[0].shape}, HR shape: {hr_images[0].shape}")
print(f"LR range: [{lr_images.min()}, {lr_images.max()}]")
print(f"HR range: [{hr_images.min()}, {hr_images.max()}]")

In [ ]:
# Visualizing Random Images
random_idx = random.randint(0, len(lr_images)-1)
plt.figure(figsize=(8,8))
plt.subplot(2, 2, 1)
plt.imshow(bgr_to_rgb(lr_images[random_idx]))
plt.title(" LR Sample ")
plt.axis("off")
plt.subplot(2, 2, 2)
plt.imshow(bgr_to_rgb(hr_images[random_idx]))
plt.title(" HR Sample ")
plt.axis("off")
plt.show()

In [ ]:
# Train-test split
lr_train, lr_test, hr_train, hr_test = train_test_split(lr_images, hr_images, test_size=0.33, random_state=42)
hr_shape = (256,256,3)
lr_shape = (64,64,3)

In [ ]:
# --- Model Initialization ---
# Initialize models
generator = srgan_generator()
generator.summary()

In [ ]:
discriminator = srgan_discriminator()
discriminator.summary()

In [ ]:
vgg = build_vgg() 
vgg.trainable = False

In [ ]:
# Training Parameters
batch_size = 6
epochs = 100  # Adjust as needed
train_dataset = tf.data.Dataset.from_tensor_slices((lr_train, hr_train)).batch(batch_size)

In [ ]:
# Loss tracking lists
gen_losses = []
disc_losses = []

In [ ]:

# Training Loop
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    
    # Initialize loss for this epoch
    epoch_gen_loss = 0
    epoch_disc_loss = 0
    
    # Add tqdm for progress bar in the batch loop
    for lr_batch, hr_batch in tqdm(tf.data.Dataset.from_tensor_slices((lr_train, hr_train)).batch(batch_size), 
                                    total=len(lr_train) // batch_size, desc=f"Training Epoch {epoch+1}/{epochs}"):
        # Discriminator Training
        with tf.GradientTape() as disc_tape:
            fake_hr = generator(lr_batch)
            real_output = discriminator(hr_batch)
            fake_output = discriminator(fake_hr)
            
            # Hinge loss
            disc_loss = tf.reduce_mean(tf.nn.relu(1.0 - real_output)) + \
                       tf.reduce_mean(tf.nn.relu(1.0 + fake_output))
        
        disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        disc_optimizer.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
        
        # Generator Training
        with tf.GradientTape() as gen_tape:
            fake_hr = generator(lr_batch)
            gen_features = vgg(fake_hr)
            real_features = vgg(hr_batch)
            
            # Content loss (MSE in feature space)
            content_loss = tf.reduce_mean(tf.abs(gen_features - real_features))
            
            # Adversarial loss (Discriminator feedback)
            adv_loss = -tf.reduce_mean(discriminator(fake_hr))
            
            gen_loss = content_loss + 1e-3 * adv_loss  # Weights for balancing content vs. adversarial loss
        
        gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gen_optimizer.apply_gradients(zip(gen_grads, generator.trainable_variables))
        
        # Accumulate losses
        epoch_gen_loss += gen_loss
        epoch_disc_loss += disc_loss
    
    # Append average loss for this epoch
    gen_losses.append(epoch_gen_loss / len(lr_train))
    disc_losses.append(epoch_disc_loss / len(lr_train))
    
    # Optionally: Print out the average losses per epoch
    print(f"Epoch {epoch+1} - Generator Loss: {gen_losses[-1]}, Discriminator Loss: {disc_losses[-1]}")

# Save final generator
generator.save("srgan_generator_final.h5")

In [ ]:
generator.save("srgan_generator_final.h5")

In [ ]:
# Evaluation Metrics
def evaluate_gan(generator, lr_test, hr_test, batch_size=8):
    psnr_values = []
    ssim_values = []
    mse_values = []
    lr_samples = []
    hr_samples = []
    sr_samples = []
    
    for i in tqdm(range(0, len(lr_test), batch_size), desc="Generating SR images"):
        lr_batch = lr_test[i:i+batch_size]
        hr_batch = hr_test[i:i+batch_size]
        
        sr_batch = generator.predict(lr_batch, verbose=0)
        sr_batch = np.clip(sr_batch, 0.0, 1.0)
        
        for j in range(len(hr_batch)):
            psnr_val = psnr(hr_batch[j], sr_batch[j], data_range=1.0)
            ssim_val = ssim(hr_batch[j], sr_batch[j], data_range=1.0, channel_axis=-1)
            mse_val = np.mean((hr_batch[j] - sr_batch[j]) ** 2)
            
            psnr_values.append(psnr_val)
            ssim_values.append(ssim_val)
            mse_values.append(mse_val)
        
        lr_samples.extend(lr_batch)
        hr_samples.extend(hr_batch)
        sr_samples.extend(sr_batch)
    
    return {
        'psnr': psnr_values,
        'ssim': ssim_values,
        'mse': mse_values,
        'lr_samples': np.array(lr_samples),
        'hr_samples': np.array(hr_samples),
        'sr_samples': np.array(sr_samples)
    }

gan_metrics = evaluate_gan(generator, lr_test, hr_test)
print("Mean PSNR:", np.mean(gan_metrics['psnr']))
print("Mean SSIM:", np.mean(gan_metrics['ssim']))
print("Mean MSE: ", np.mean(gan_metrics['mse']))

In [ ]:
# Loss Visualization
plt.figure(figsize=(8, 6))
plt.plot(range(1, epochs+1), gen_losses, color='blue', marker='o', label='Generator Loss')
plt.title('Generator Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(range(1, epochs+1), disc_losses, color='orange', marker='o', label='Discriminator Loss')
plt.title('Discriminator Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Sample Comparisons
def plot_gan_samples(gan_metrics, num_samples=3):
    indices = np.random.choice(len(gan_metrics['lr_samples']), num_samples)
    for idx in indices:
        plt.figure(figsize=(12,6))
        # LR Input (BGR -> RGB for display)
        plt.subplot(241)
        lr_bgr = gan_metrics['lr_samples'][idx]
        plt.imshow(bgr_to_rgb(lr_bgr))
        plt.title('LR Input')
        plt.axis('off')
        
        # HR Target (BGR -> RGB for display)
        plt.subplot(242)
        hr_bgr = gan_metrics['hr_samples'][idx]
        plt.imshow(bgr_to_rgb(hr_bgr))
        plt.title('HR Target')
        plt.axis('off')
        
        # SR Output (BGR -> RGB for display)
        plt.subplot(243)
        sr_bgr = gan_metrics['sr_samples'][idx]
        metrics_text = f"PSNR: {gan_metrics['psnr'][idx]:.2f}\nSSIM: {gan_metrics['ssim'][idx]:.4f}\nMSE: {gan_metrics['mse'][idx]:.5f}"
        plt.imshow(bgr_to_rgb(sr_bgr))
        plt.title('SRGAN Output\n' + metrics_text)
        plt.axis('off')
        
        # Error Map
        plt.subplot(244)
        error = np.abs(hr_bgr - sr_bgr)
        plt.imshow(np.mean(error, axis=-1), cmap='inferno', vmin=0, vmax=0.07)
        plt.title('Pixel Error Heatmap')
        plt.colorbar()
        plt.axis('off')
        plt.tight_layout()
        plt.show()

plot_gan_samples(gan_metrics)

In [ ]:
# Metric Distributions
def plot_gan_metrics(gan_metrics):
    fig, axs = plt.subplots(1, 3, figsize=(18,5))
    metrics = ['psnr', 'ssim', 'mse']
    titles = ['PSNR Distribution', 'SSIM Distribution', 'MSE Distribution']
    colors = ['purple', 'green', 'red']
    for ax, metric, title, color in zip(axs, metrics, titles, colors):
        ax.hist(gan_metrics[metric], bins=30, alpha=0.7, color=color)
        ax.set_title(title)
        ax.set_xlabel(metric.upper())
        ax.set_ylabel('Frequency')
        mean_val = np.mean(gan_metrics[metric])
        ax.axvline(mean_val, color='black', linestyle='dashed', linewidth=2, label=f'Mean: {mean_val:.2f}')
        ax.legend()
    plt.tight_layout()
    plt.show()

plot_gan_metrics(gan_metrics)

In [ ]:
# Error Analysis
def plot_gan_error_analysis(gan_metrics):
    fig, axs = plt.subplots(1, 2, figsize=(12,5))
    errors = np.concatenate([np.abs(hr - sr) for hr, sr in zip(gan_metrics['hr_samples'], gan_metrics['sr_samples'])])
    axs[0].hist(errors.flatten(), bins=50, color='darkorange', density=True)
    axs[0].set_title('Pixel Error Distribution')
    axs[0].set_xlabel('Absolute Error')
    axs[0].set_ylabel('Density')
    
    axs[1].scatter(gan_metrics['psnr'], gan_metrics['ssim'], alpha=0.5)
    axs[1].set_title('PSNR vs SSIM Correlation')
    axs[1].set_xlabel('PSNR')
    axs[1].set_ylabel('SSIM')
    
    psnr_data = np.array(gan_metrics['psnr'])
    ssim_data = np.array(gan_metrics['ssim'])
    X = psnr_data.reshape(-1, 1)
    y = ssim_data
    m, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    x_line = np.linspace(0, psnr_data.max(), 100)
    y_line = m[0] * x_line
    axs[1].plot(x_line, y_line, color='red', linestyle='--', label=f'Trend: y = {m[0]:.5f}x')
    axs[1].legend()
    plt.tight_layout()
    plt.show()

plot_gan_error_analysis(gan_metrics)

In [ ]:
#----Test with New Dataset ----

In [ ]:
# Define new test dataset directories
new_lr_dir = "/kaggle/input/test-sr/testing dataset/LR"
new_hr_dir = "/kaggle/input/test-sr/testing dataset/HR"

# Helper function to load images in RGB (for SRGAN)
def load_images_rgb(folder, resize_shape):
    filenames = sorted(os.listdir(folder))
    images = []
    for file in filenames:
        path = os.path.join(folder, file)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, resize_shape)
            images.append(img.astype(np.float32) / 255.0)
    return np.array(images)

# Load new LR and HR images for SRGAN (RGB format)
new_lr_images = load_images_rgb(new_lr_dir, (64,64))
new_hr_images = load_images_rgb(new_hr_dir, (256,256))

# Evaluate on new dataset (using your evaluate_gan function)
new_gan_metrics = evaluate_gan(generator, new_lr_images, new_hr_images)

# Print side-by-side performance comparison in text
orig_psnr = np.mean(gan_metrics['psnr'])
orig_ssim = np.mean(gan_metrics['ssim'])
orig_mse  = np.mean(gan_metrics['mse'])

new_psnr = np.mean(new_gan_metrics['psnr'])
new_ssim = np.mean(new_gan_metrics['ssim'])
new_mse  = np.mean(new_gan_metrics['mse'])

print("\n==================== SRGAN PERFORMANCE COMPARISON ====================")
print(f"{'Metric':<8} | {'Original':>10} | {'New':>10} | {'Diff':>10}")
print("-"*56)
for name, o_val, n_val in zip(["PSNR", "SSIM", "MSE"], [orig_psnr, orig_ssim, orig_mse], [new_psnr, new_ssim, new_mse]):
    diff = n_val - o_val
    print(f"{name:<8} | {o_val:10.4f} | {n_val:10.4f} | {diff:10.4f}")

In [ ]:
# Plot Bar Charts

plt.figure(figsize=(12,5))
metrics = ["PSNR","SSIM","MSE"]
orig_data = [orig_psnr, orig_ssim, orig_mse]
new_data  = [new_psnr, new_ssim, new_mse]
colors = ["blue","orange"]

for i, (metric, orig_val, new_val) in enumerate(zip(metrics, orig_data, new_data)):
    plt.subplot(1,3,i+1)
    plt.bar(["Original","New"], [orig_val,new_val], color=colors)
    plt.title(f"{metric} Comparison")
    plt.ylabel(metric)

plt.suptitle("SRGAN Metric Comparison", fontsize=14)
plt.tight_layout()
plt.show()